# Tarjetas Black


Vamos a utilizar un dataset de movimientos de tarjetas de crédito que provienen de tarjetas Black. 

El origen es el siguiente repositorio de GitHub (aunque han sido preprocesados en R):

https://github.com/splatsh/tarjetasblack

Originalmente la información viene de la siguiente fuente:

https://www.cuartopoder.es/multimedia/2014/10/11/gastos-de-los-exdirectivos-de-caja-madrid-uno-a-uno-con-las-tarjetas-negras-tabla/3403


## Modelo conceptual

<br><br> 


<img src="images/Modelo%20conceptual.png",width=800,height=500>
<br><br> 

## Modelo de relación

<br><br> 

<img src="images/Modelo%20relacion.png",width=500,height=500>

<br><br> 

## Contenido de la información

Contiene la siguiente información:

- Nombre: Nombre de la persona que posee la tarjeta y realiza el movimiento
- Fecha: Fecha del movimiento
- Hora: Hora del movimiento
- Minuto: Minuto del movimiento
- Importe: Importe del movimiento
- Comercio donde se realiza el movimiento
- Actividad Completa: Detalle de la que se dedica el comercio
- Actividad: Contiene una clasificación de la actividad del comercio. Una Actividad puede agrupar a varios comercios
- Función: Función que desempeña la persona dentro de su organización (Sólo para los partidos políticos)
- Organización: Organización a la que pertenece la persona que realiza el gasto

Podrían existir movimientos con campos vacíos.

## Formato del fichero

El fichero con los datos está comprimido, en formato CSV, y utiliza el carácter ~ cómo separador de columnas.


## Preguntas

Vamos a intentar resolver las siguientes preguntas:
    
- Los 10 movimientos mas caros por actividad
- Los 10 movimientos mas caros
- Los movimientos de una persona concreta (ordenados por importe)
- Las 10 personas que mas han gastado
- Importes de una persona agrupados por actividad
- ¿Quién se gasta más, los concejales o los directivos?
- ¿Qué organización se gasta más?
- ¿Qué comercio es más popular entre los miembros?

# Preprocesado de datos

Se realizará un preprocesado de datos cuyo objetico es limpiar y normalizar los datos para que se puedan utilizar más fácilmente por el resto de prácticas  

El formato de salida será un fichero Microsoft Excel que contendrá los siguientes datasets:

- Movimientos: Movimientos donde se han eliminado los atributos personales y se han sustituido por un ID
- Miembros: Dataset con todas las personas que han realzado movimientos

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../../data/black.txt.gz", compression='gzip', sep='~',  index_col=False )

In [3]:
df.head()

,nombre,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,funcion,organizacion
0,Alberto Recarte García Andrade,2003-01-03,12,30,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA,concejal,Partido Popular
1,Alberto Recarte García Andrade,2003-01-03,12,32,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL,concejal,Partido Popular
2,Alberto Recarte García Andrade,2003-01-04,19,7,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE,concejal,Partido Popular
3,Alberto Recarte García Andrade,2003-01-07,15,31,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE,concejal,Partido Popular
4,Alberto Recarte García Andrade,2003-01-07,16,5,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE,concejal,Partido Popular


Nos quedamos sólo con los registros únicos

In [4]:
df_miembros = df[["nombre", "funcion", "organizacion"]].drop_duplicates()
df_miembros.head()

,nombre,funcion,organizacion
0,Alberto Recarte García Andrade,concejal,Partido Popular
1305,Alejandro Couceiro Ojeda,concejal,CEIM
1851,Ángel Eugenio Gómez del Pulgar Perales,concejal,PSOE
2832,Angel Rizaldos González,concejal,Izquierda Unida
2952,Antonio Cámara Eguinoa,concejal,Partido Popular


Se crea un dataset de miembros, asignado a cada persona un ID único

In [5]:
df_miembros = df_miembros.assign(id_miembro = df_miembros.nombre.rank(method='min').astype(int))
df_miembros = df_miembros[['id_miembro', 'nombre', 'funcion', 'organizacion']]
df_miembros.head()

,id_miembro,nombre,funcion,organizacion
0,1,Alberto Recarte García Andrade,concejal,Partido Popular
1305,2,Alejandro Couceiro Ojeda,concejal,CEIM
1851,83,Ángel Eugenio Gómez del Pulgar Perales,concejal,PSOE
2832,3,Angel Rizaldos González,concejal,Izquierda Unida
2952,4,Antonio Cámara Eguinoa,concejal,Partido Popular


Se inserta en el dataset principal el ID creado anteriormente

In [6]:
df_result = pd.merge(df, df_miembros, on = ['nombre', 'funcion', 'organizacion'], how = 'inner')
df_result.head()

,nombre,fecha,hora,minuto,importe,comercio,actividad_completa,actividad,funcion,organizacion,id_miembro
0,Alberto Recarte García Andrade,2003-01-03,12,30,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA,concejal,Partido Popular,1
1,Alberto Recarte García Andrade,2003-01-03,12,32,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL,concejal,Partido Popular,1
2,Alberto Recarte García Andrade,2003-01-04,19,7,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE,concejal,Partido Popular,1
3,Alberto Recarte García Andrade,2003-01-07,15,31,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE,concejal,Partido Popular,1
4,Alberto Recarte García Andrade,2003-01-07,16,5,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE,concejal,Partido Popular,1


Seleccionamos las columnas que nos interesan

In [7]:
df_movimientos = df_result[['id_miembro','fecha', 'minuto', 'hora', 'importe', 'comercio', 'actividad_completa', 'actividad']]

Eliminamos la información horaria de la fecha

In [8]:
df_movimientos = df_movimientos.assign(fecha = pd.to_datetime(df['fecha']).dt.date)
df_movimientos.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 76238 entries, 0 to 76237
Data columns (total 8 columns):
id_miembro            76238 non-null int64
fecha                 76238 non-null object
minuto                76238 non-null int64
hora                  76238 non-null int64
importe               76238 non-null float64
comercio              68500 non-null object
actividad_completa    76221 non-null object
actividad             76220 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 5.2+ MB


In [9]:
df_movimientos.head()

,id_miembro,fecha,minuto,hora,importe,comercio,actividad_completa,actividad
0,1,2003-01-03,30,12,38.70,RCG OFICINA,CONFECCION TEXTIL EN GENERAL,ROPA
1,1,2003-01-03,32,12,14.60,MANZANIL AREA,"HOTELES,MOTELES,BALNEARIOS,CAMPINGS REST",HOTEL
2,1,2003-01-04,7,19,95.62,REST REAL C GOLF SOTOGRAN,RESTAURANTES RESTO,RESTAURANTE
3,1,2003-01-07,31,15,49.13,ESTACONES DE SERVICIO ML,GASOLINERAS,COCHE
4,1,2003-01-07,5,16,13.93,NISA (H 24) VINHOLAN S.A.,GASOLINERAS,COCHE


Y por último selecionamos al azar un 10% de los datos para que la práctica sea más ágil ...

In [10]:
df_movimientos_lite = df_movimientos.sample(frac=0.10)
df_movimientos_lite.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7624 entries, 30053 to 21461
Data columns (total 8 columns):
id_miembro            7624 non-null int64
fecha                 7624 non-null object
minuto                7624 non-null int64
hora                  7624 non-null int64
importe               7624 non-null float64
comercio              6864 non-null object
actividad_completa    7622 non-null object
actividad             7622 non-null object
dtypes: float64(1), int64(3), object(4)
memory usage: 536.1+ KB


In [11]:
df_movimientos_lite.head()

,id_miembro,fecha,minuto,hora,importe,comercio,actividad_completa,actividad
30053,40,2005-03-28,11,16,73.00,LA VACA ARGENTINA,RESTAURANTES RESTO,RESTAURANTE
55589,80,2006-06-27,4,15,28.78,ASADOR CRUZ NEVADA,RESTAURANTES RESTO,RESTAURANTE
56355,80,2008-07-21,40,14,1.79,ACCESOS DE MADRID,AUTOPISTAS (TERMINALES),COCHE
14018,16,2007-01-14,1,16,29.69,OAM THONG,RESTAURANTES RESTO,RESTAURANTE
63577,61,2004-03-11,7,17,519.60,RESTAURANTE HORCHER,RESTAURANTE 4/5 TENEDORES NIVEL 2,RESTAURANTE


Grabamos el resultado en un fichero Excel donde cada dataset estará en un hoja diferente

In [12]:
writer = pd.ExcelWriter("../../data/black.xlsx")
df_movimientos_lite.to_excel(writer,'Movimientos')
df_miembros.to_excel(writer, 'Miembros')
writer.save()